In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
[1,2,3,4,5,6,7,8][2:]

[3, 4, 5, 6, 7, 8]

In [3]:
import requests
from bs4 import BeautifulSoup
from pythainlp import thai_consonants

In [4]:
headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"}

In [ ]:
# table law
# sysid auto increment
# title text
# txt longtext

In [5]:
import mysql.connector

cnx = mysql.connector.connect(user='', password='',host='127.0.0.1',database='law3') # user password

In [6]:
def add_data(sysid,title,txt):
    cnx2 = mysql.connector.connect(user='root', password='',host='127.0.0.1',database='law3')
    add_db = "insert into law(sysid,title,txt) values(%s,%s,%s);"
    cursor = cnx2.cursor()
    cursor.execute(add_db,(str(sysid),str(title),str(txt)))
    cnx2.commit()
    cnx2.close()

In [7]:
def check_db(sysid):
    cnx3 = mysql.connector.connect(user='root', password='',host='127.0.0.1',database='law3')
    cursor = cnx3.cursor()
    sql = ('select sysid from law where sysid=%s;')
    #print(sql)
    cursor.execute(sql,[str(sysid)])
    check = cursor.fetchall() != []
    cnx3.close()
    return check

In [8]:
def geturl(c):
    return 'https://www.krisdika.go.th/web/guest/law?p_p_id=LawPortlet_INSTANCE_aAN7C2U5hENi&p_p_lifecycle=2&p_p_state=normal&p_p_mode=view&p_p_cacheability=cacheLevelPage&_LawPortlet_INSTANCE_aAN7C2U5hENi_lawShowType=showAll&p_p_lifecycle=1&_LawPortlet_INSTANCE_aAN7C2U5hENi_javax.portlet.action=selectLawShowType&_LawPortlet_INSTANCE_aAN7C2U5hENi_character={c}&_LawPortlet_INSTANCE_aAN7C2U5hENi_groupOfAct=byCharacter&_LawPortlet_INSTANCE_aAN7C2U5hENi_groupThcaId=0&_LawPortlet_INSTANCE_aAN7C2U5hENi_lawTypeId=2'.format(c=c)
    #return "https://www.krisdika.go.th/web/guest/law?p_p_id=LawPortlet_INSTANCE_aAN7C2U5hENi&p_p_lifecycle=2&p_p_state=normal&p_p_mode=view&p_p_cacheability=cacheLevelPage&_LawPortlet_INSTANCE_aAN7C2U5hENi_lawShowType=showAll&p_p_lifecycle=1&_LawPortlet_INSTANCE_aAN7C2U5hENi_javax.portlet.action=selectLawShowType&_LawPortlet_INSTANCE_aAN7C2U5hENi_character={c}&_LawPortlet_INSTANCE_aAN7C2U5hENi_groupOfAct=byCharacter&_LawPortlet_INSTANCE_aAN7C2U5hENi_groupThcaId=0&_LawPortlet_INSTANCE_aAN7C2U5hENi_lawTypeId=2"

In [9]:
def get_json(c):
    u = geturl(c)
    r = requests.get(u, verify=False,headers=headers)
    return r.json()

In [10]:
def get_sysid(v):
    if "&ext=htm" not in v:
        return None
    return v.split('&ext=htm')[0].split('https://www.krisdika.go.th/librarian/get?sysid=')[1]

In [11]:
def gethtml(sysid):
    return "https://www.krisdika.go.th/librarian/getfile?sysid={sysid}&ext=htm".format(sysid=sysid)

In [12]:
def get_data(sysid):
    r2 = requests.get(gethtml(sysid), verify=False,headers=headers)
    d = r2.content.decode("TIS-620")
    soup = BeautifulSoup(d,'html.parser')
    return soup.text

In [15]:
#get_json('ง')

In [16]:
from tqdm.notebook import trange, tqdm

In [17]:
from concurrent.futures import ThreadPoolExecutor

In [18]:
executor = ThreadPoolExecutor(max_workers=5)

In [19]:
def savetodb(o):
    try:
        sysid = get_sysid(o['value'])
        if check_db(sysid):
            pass
        else:
            txt = get_data(sysid)
            title = BeautifulSoup(o['value'],'html.parser').text
            add_data(sysid,title,txt)
    except:
        pass

In [20]:
num=0
def save(c):
    j = get_json(c)
    if j == {'rows': []}:
        pass
    for item in j['rows']:
        if  item == {'rows': []}:
            continue
        for i in item['rows']:
            if  i== {'rows': []}:
                continue
            #print(i)
            if 'rows' not in i.keys() and 'data' in i.keys():
                for o in i['data']:
                    executor.submit(savetodb,o)
                continue
            for j in i['rows']:
                if  j == {'rows': []}:
                    continue
                for o in j['data']:
                    executor.submit(savetodb,o)

In [24]:
for i in thai_consonants:
    try:
        save(i)
    except:
        pass

In [ ]:
30 # เสร็จ 31 ข้าม

In [23]:
thai_consonants[1:]

'ขฃคฅฆงจฉชซฌญฎฏฐฑฒณดตถทธนบปผฝพฟภมยรลวศษสหฬอฮ'

In [ ]:
thai_consonants[31]

In [ ]:
#save('ข')

In [ ]:
#save('พ')